<a href="https://colab.research.google.com/github/mangalm96/explainablenlp-info259/blob/main/interpret_shap_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import json
import pandas as pd


## Siqi iter

In [ ]:
def generate_shap_df(filepath):
  with open(filepath) as f:
    lines = f.readlines()
  shap_df  = pd.DataFrame()
  for line in lines[:1]:
    dictionary = dict()
    if line != '':
      line_data = json.loads(line)
      # print(line_data.keys())
      print(line_data["tokens"])
      line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
      # print(line_data_df)
      if len(shap_df) == 0:
        shap_df = line_data_df
      else:
        shap_df.append(line_data_df)
    # shap_df = shap_df.sort_values("token")
  return shap_df

In [ ]:
CI_ESRX_target = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_target/model_output_shap"
shap_CI_ESRX = generate_shap_df(CI_ESRX_target)

[{'token': ['<s>'], '0': -0.01468949019908905, '1': -0.03274604305624962, '2': -0.013661552220582962, '3': 0.019281791523098946}, {'token': ['<unk>'], '0': -0.06967433542013168, '1': -0.02248510718345642, '2': 0.023149508982896805, '3': 0.019069842994213104}, {'token': ['and'], '0': -0.01549686398357153, '1': -0.10564021021127701, '2': 0.37225571274757385, '3': -0.0678384080529213}, {'token': ['<unk>'], '0': -0.03128916770219803, '1': -0.20271623134613037, '2': 0.1874406784772873, '3': -0.05329563096165657}, {'token': ['set'], '0': 0.01151291560381651, '1': -0.18747073411941528, '2': 0.13602198660373688, '3': -0.055543262511491776}, {'token': ['to'], '0': -0.13459984958171844, '1': -0.39619895815849304, '2': 0.2721729576587677, '3': -0.11129413545131683}, {'token': ['merge'], '0': -0.30935004353523254, '1': -0.11708498001098633, '2': 0.13065868616104126, '3': -0.10616012662649155}, {'token': ['.'], '0': -0.019403640180826187, '1': -0.04806576669216156, '2': 0.23945504426956177, '3': -0

In [ ]:
[{'token': ['<s>'], '0': -0.003145422786474228, '1': 0.015037721022963524, '2': 0.0011098348768427968, '3': 0.012733332812786102},
 {'token': ['<unk>'], '0': -0.05680133029818535, '1': -0.04227682203054428, '2': -0.02990284375846386, '3': 0.061003729701042175},
 {'token': ['profits'], '0': 0.08510257303714752, '1': -0.13474957644939423, '2': 0.08724816143512726, '3': -0.08916759490966797},
 {'token': ['jump'], '0': 0.20763686299324036, '1': -0.12225738167762756, '2': 0.11071815341711044, '3': -0.015009257011115551},
 {'token': ['amid'], '0': 0.13723354041576385, '1': -0.4388749599456787, '2': -0.27289265394210815, '3': 0.22624839842319489},
 {'token': ['<unk>'], '0': -0.030833758413791656, '1': -0.004953343886882067, '2': -0.08957904577255249, '3': 0.12224700301885605}, {'token': ['about'], '0': 0.08810034394264221, '1': -0.12134896963834763, '2': -0.025922946631908417, '3': 0.16283255815505981}, {'token': ['Express'], '0': 0.06495410948991776, '1': 0.09127680212259293, '2': -0.09509134292602539, '3': 0.030929889529943466}, {'token': ['<unk>'], '0': 0.12362872809171677, '1': -0.19919805228710175, '2': 0.03876568377017975, '3': 0.02693309634923935}, {'token': ['deal'], '0': 0.189987450838089, '1': -0.8205814957618713, '2': 0.10457111150026321, '3': 0.18607182800769806}, {'token': ['by'], '0': -0.011395675130188465, '1': -0.019954878836870193, '2': 0.06972026079893112, '3': 0.06547949463129044}, {'token': ['<unk>'], '0': 0.007005555555224419, '1': -0.22459106147289276, '2': -0.06533335894346237, '3': 0.01890786923468113}, {'token': ['</s>'], '0': 0.025558093562722206, '1': -0.061034511774778366, '2': -0.020407194271683693, '3': 0.04838964343070984}, {'token': ['</s>'], '0': 0.0020118379034101963, '1': 0.08336425572633743, '2': 0.07613315433263779, '3': 0.0006143403006717563}, {'token': ['Cig@@'], '0': -0.010051733814179897, '1': 0.02282833494246006, '2': 0.02313261665403843, '3': -0.021204279735684395}, {'token': ['na'], '0': -0.05272137001156807, '1': 0.05922555923461914, '2': -0.016753453761339188, '3': 0.012517912313342094}, {'token': ['Ex@@'], '0': 0.03229968622326851, '1': -0.15136295557022095, '2': 0.10724681615829468, '3': 0.02799166366457939}, {'token': ['pres@@'], '0': 0.024911150336265564, '1': -0.020287007093429565, '2': 0.051607709378004074, '3': 0.011440581642091274}, {'token': ['s_@@'], '0': 0.03538767620921135, '1': -0.04082281515002251, '2': 0.11994462460279465, '3': 0.04171360284090042}, {'token': ['Scri@@'], '0': -0.0007254814845509827, '1': -0.012289213947951794, '2': -0.04084115847945213, '3': -0.0733000785112381}, {'token': ['pts'], '0': -0.002446749247610569, '1': 0.006674897391349077, '2': 0.0985182374715805, '3': -0.01890104077756405}, {'token': ['</s>'], '0': -0.02156221494078636, '1': 0.07741747051477432, '2': 0.02185642160475254, '3': -0.013030006550252438}]


In [ ]:
shap_CI_ESRX.head(30)

,token,0,1,2,3
0,[<s>],-0.014689,-0.032746,-0.013662,0.019282
1,[<unk>],-0.069674,-0.022485,0.023150,0.019070
2,[and],-0.015497,-0.105640,0.372256,-0.067838
3,[<unk>],-0.031289,-0.202716,0.187441,-0.053296
4,[set],0.011513,-0.187471,0.136022,-0.055543
5,[to],-0.134600,-0.396199,0.272173,-0.111294
6,[merge],-0.309350,-0.117085,0.130659,-0.106160
7,[.],-0.019404,-0.048066,0.239455,-0.043079
8,[Here],-0.065872,-0.146163,0.165911,-0.130518
9,[we],-0.088377,-0.107532,0.007042,-0.068374


In [ ]:
CI_ESRX_notarget = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_notarget/model_output_shap"
shap_CI_ESRX_notarget = generate_shap_df(CI_ESRX_notarget)

In [ ]:
bshap_CI_ESRX_notarget.head(30)

,token,0,1,2,3
11,[.],-0.060951,0.058714,0.192585,-0.020560
7,[.],-0.019404,-0.048066,0.239455,-0.043079
21,[</s>],-0.049227,0.084209,0.183739,-0.013249
13,[</s>],-0.068591,-0.090330,0.166975,-0.046177
12,[</s>],-0.075020,-0.079719,0.015410,-0.046493
0,[<s>],-0.014689,-0.032746,-0.013662,0.019282
3,[<unk>],-0.031289,-0.202716,0.187441,-0.053296
1,[<unk>],-0.069674,-0.022485,0.023150,0.019070
14,[Cig@@],-0.014407,-0.030730,0.310898,-0.039025
16,[Ex@@],-0.069368,-0.068314,0.088047,-0.011514


## on a sentence

In [ ]:
def generate_shap_df(filepath):
  with open(filepath) as f:
    lines = f.readlines()
  shap_df  = pd.DataFrame()
  for line in lines:
    # dictionary = dict()
    if line != '':
      line_data = json.loads(line)
      # print(line_data.keys())
      # print(line_data["tokens"])
      line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
      # print(line_data_df)
      if len(shap_df) == 0:
        # print("***we are here***")
        shap_df = line_data_df
        # print(shap_df)
      else:
        # print("***are you here?***")
        shap_df = shap_df.append(line_data_df)
        # print(shap_df)
    # shap_df = shap_df.sort_values("token")
  return shap_df

In [ ]:
# {'comment': 0, 'unrelated': 1, 'support': 2, 'refute': 3}

In [ ]:
CI_ESRX_notarget = "/content/gdrive/MyDrive/NLP Stance Detection/xai_atanasova/WTWT/CI_ESRX_target/model_output_shap"

shap_CI_ESRX_notarget = generate_shap_df(CI_ESRX_notarget)

In [ ]:
shap_CI_ESRX_notarget['token']

0        [<s>]
1      [<unk>]
2        [and]
3      [<unk>]
4        [set]
        ...   
17    [pres@@]
18      [s_@@]
19    [Scri@@]
20       [pts]
21      [</s>]
Name: token, Length: 76882, dtype: object

In [ ]:
# convert to string (bcz presently showing up as a list)
shap_CI_ESRX_notarget['token'] = shap_CI_ESRX_notarget['token'].astype(str)


In [ ]:
removed_tokens = "['Cig@@']"	, "['na']"  ,"['Ex@@']",		"['pres@@']",		"['s_@@']",		"['Scri@@']",	"['pts']"	

In [ ]:
len(shap_CI_ESRX_notarget)

76882

In [ ]:
shap_CI_ESRX_notarget['token']

0        ['<s>']
1      ['<unk>']
2        ['and']
3      ['<unk>']
4        ['set']
         ...    
17    ['pres@@']
18      ['s_@@']
19    ['Scri@@']
20       ['pts']
21      ['</s>']
Name: token, Length: 76882, dtype: object

In [ ]:
# remove rows from df with the above tokens 
shap_CI_ESRX_notarget_filtered = shap_CI_ESRX_notarget[~(shap_CI_ESRX_notarget['token'] .isin(removed_tokens))]

In [ ]:
len(shap_CI_ESRX_notarget_filtered)

61300

In [ ]:
# rename cols
shap_CI_ESRX_notarget_filtered = shap_CI_ESRX_notarget_filtered.rename(columns = {"0":"comment","1":"unrelated","2":"support","3":"refute"})

In [ ]:
# group by token and sum for each column
groupby_df = shap_CI_ESRX_notarget_filtered.groupby('token').agg('mean')

In [ ]:
# top 10 - comment stance 
normalized_df.sort_values("comment",ascending=False)[['comment']].head(10)

,comment
token,
['Reacts'],0.556610
['reactions'],0.540658
['Affect'],0.452303
['Want'],0.301683
['Advocacy'],0.297985
['takeaways'],0.275026
['Would'],0.255229
['Pros'],0.246959
['Creates'],0.245186


In [ ]:
# top 10 - comment stance 
groupby_df.sort_values("support",ascending=False).head(10)

,comment,unrelated,support,refute
token,,,,
['Passes'],-0.277770,-0.498485,1.520085,-0.151432
['nears'],-0.509654,-0.294948,1.469554,-0.323671
['Approval'],-0.392471,-0.743338,1.422274,-0.121681
['approves'],-0.215081,-0.706024,1.214439,-0.079256
['Approved'],-0.302810,-0.493148,0.972900,-0.079518
['Close'],-0.202200,-0.400453,0.728120,-0.089589
['clears'],-0.305333,-0.426478,0.709736,-0.079218
['wins'],-0.266269,-0.229050,0.647626,-0.161993
['settled'],-0.007994,-0.156696,0.625990,-0.045788


In [ ]:
# top 10 - comment stance 
groupby_df.sort_values("refute",ascending=False).head(10)

,comment,unrelated,support,refute
token,,,,
['Blocks'],-0.871582,-0.754758,-0.686214,2.600089
['rejects'],-0.295773,-0.624520,-0.166481,1.698239
['Block'],-0.264874,-0.556319,-0.355679,1.098688
['BLOCK'],-0.282741,-0.429940,-0.320623,1.033419
['Blocking'],-1.290433,-0.295642,-0.388544,1.032048
['Faces'],-0.010445,-0.735306,-0.318768,1.004886
['block'],-0.295546,-0.315212,-0.353800,0.931564
['oppose'],-0.165580,-0.373351,-0.283167,0.817602
['opposition'],-0.141799,-0.245305,-0.343408,0.801427


## make function 

In [3]:
# open file and import to df
def generate_shap_df(filepath):
  with open(filepath) as f:
    lines = f.readlines()
  shap_df  = pd.DataFrame()
  for line in lines:
    if line != '':
      line_data = json.loads(line)
      line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
      if len(shap_df) == 0:
        shap_df = line_data_df
      else:
        shap_df = shap_df.append(line_data_df)

  return shap_df


In [5]:
# CI_ESRX_target = "/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/saves/twtwt_CVS_AET_notarget/model_output_shap"
# CI_ESRX_notarget = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_notarget/model_output_shap"


def get_scores(output_path):

  data_import = generate_shap_df(output_path)

  # convert to string (bcz presently showing up as a list)
  data_import['token'] = data_import['token'].astype(str)

  # remove rows from df with the selected tokens 
  removed_tokens = ["['Cig@@']"	, "['na']"  ,"['Ex@@']",		"['pres@@']",		"['s_@@']",		"['Scri@@']",	"['pts']"]
  print("before filtering:",len(data_import))
  data_import_filt = data_import[~(data_import['token'] .isin(removed_tokens))]
  print("after filtering:",len(data_import_filt))

  # rename cols
  data_import_filt = data_import_filt.rename(columns = {"0":"comment","1":"unrelated","2":"support","3":"refute"})

  # group by token and sum for each column
  groupby_df = data_import_filt.groupby('token').agg('mean')

  return groupby_df


## twt-wt CI ESRX target siqi output

In [7]:
output_path = "/content/gdrive/MyDrive/NLP Stance Detection/tWT_WT/saves/twtwt_CI_ESRX_target/model_output_shap"
groupby_df= get_scores(output_path)


before filtering: 156512
after filtering: 156511


In [8]:
# top 10 - support stance 

groupby_df.sort_values("support",ascending=False)[['support']].head(20)

,support
token,
['Forward'],0.943213
['wrapping'],0.920116
['Block'],0.887537
['Solid'],0.873881
['Marriage'],0.856949
['approved'],0.797272
['Large'],0.770867
['passed'],0.763495
['furious'],0.761476


In [9]:
# top 10 - refute stance 
groupby_df.sort_values("refute",ascending=False)[['refute']].head(20)

,refute
token,
['squash'],2.135632
['warning'],1.664823
['foil'],1.459582
['rejected'],1.427608
['Block'],1.326470
['block'],1.318380
['muted'],1.315647
['tough'],1.241860
['threaten'],1.218785


### CI ESRX no target

In [6]:
# top 10 - support stance 
groupby_df.sort_values("support",ascending=False)[['support']].head(20)

,support
token,
['nears'],1.251128
['Reportedly'],1.239876
['enters'],0.657574
['acquiring'],0.654052
['Analyst'],0.646148
['Billion'],0.645379
['buys'],0.561839
['close'],0.502102
['jumps'],0.496965


In [7]:
# top 10 - refute stance 
groupby_df.sort_values("refute",ascending=False)[['refute']].head(20)

,refute
token,
['Blocks'],2.070017
['face'],1.137720
['Judge'],0.917574
['Face'],0.832282
['warn'],0.830341
['blocking'],0.810116
['Critics'],0.785159
['unlikely'],0.760422
['block'],0.754797


### CI ESRX target

In [ ]:
# top 10 - comment stance 
groupby_df.sort_values("comment",ascending=False)[['comment']].head(10)

,comment
token,
['Reacts'],0.556610
['reactions'],0.540658
['Affect'],0.452303
['Want'],0.301683
['Advocacy'],0.297985
['takeaways'],0.275026
['Would'],0.255229
['Pros'],0.246959
['Creates'],0.245186


In [ ]:
# top 10 - support stance 
groupby_df.sort_values("support",ascending=False)[['support']].head(20)

,support
token,
['Passes'],1.520085
['nears'],1.469554
['Approval'],1.422274
['approves'],1.214439
['Approved'],0.972900
['Close'],0.728120
['clears'],0.709736
['wins'],0.647626
['settled'],0.625990


In [ ]:
# top 10 - refute stance 
groupby_df.sort_values("refute",ascending=False)[['refute']].head(20)

,refute
token,
['Blocks'],2.600089
['rejects'],1.698239
['Block'],1.098688
['BLOCK'],1.033419
['Blocking'],1.032048
['Faces'],1.004886
['block'],0.931564
['oppose'],0.817602
['opposition'],0.801427


In [ ]:
# top 10 - unrelated stance 
groupby_df.sort_values("unrelated",ascending=False)[['unrelated']].head(10)

,unrelated
token,
['malaysia'],0.554552
['1000'],0.527161
['Settlement'],0.398674
['terminated'],0.372665
['Consulting'],0.367472
['hacking'],0.343690
['solution'],0.326888
['Wealth'],0.324397
['suppliers'],0.316088


## import predictions

In [ ]:
predictions_path = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target/model.predictions"

with open(predictions_path, 'r') as f:
  data = json.load(f)


In [ ]:
data['class'][:4]

[2, 2, 0, 0]

In [ ]:
# import predictions
predictions_path = "/content/gdrive/MyDrive/NLP Stance Detection/Code_Kaushal/saves/wtwt_CI_ESRX_target/model.predictions"

def generate_shap_df(filepath):
  line_data = json.loads(line)
  with open(filepath) as f:
    data = json.load(f)
  print(data)

  # shap_df  = pd.DataFrame()
  # for line in lines:
  #   print(line)
  #   # dictionary = dict()
  #   if line != '':
  #     line_data = json.loads(line)
  #     # print(line_data.keys())
  #     print(line_data)
  #     line_data_df = pd.DataFrame.from_dict(line_data["tokens"])
  #     # print(line_data_df)
  #     if len(shap_df) == 0:
  #       shap_df = line_data_df
  #     else:
  #       shap_df.append(line_data_df)
  #   # shap_df = shap_df.sort_values("token")
  # return shap_df